In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

/home/joaquin/Documents/GitHub/skforecast


In [2]:
from sklearn.preprocessing import OrdinalEncoder
from skforecast.preprocessing import FastOrdinalEncoder

In [3]:
products = [f"series_{i}" for i in range(1, 5001)]
data = {
    'product': np.repeat(products, 1000),
    'price': np.random.uniform(10, 100, 5000000),
}
df = pd.DataFrame(data)
display(df)

df_new_category = df.copy()
df_new_category['product'] = df_new_category['product'].replace('series_1', 'new_series')

,product,price
0,series_1,84.074753
1,series_1,73.683315
2,series_1,96.869594
3,series_1,12.184607
4,series_1,51.630885
...,...,...
4999995,series_5000,11.061986
4999996,series_5000,61.810475
4999997,series_5000,76.284423
4999998,series_5000,85.582163


In [4]:
# Fit and transform

In [5]:
%%timeit -r 10 -n 10
encoder = OrdinalEncoder(
    categories = 'auto',
    dtype      = int
).set_output(transform='pandas')
encoder.fit(df[['product']])
encoded_values = encoder.transform(df[['product']])

1.03 s ± 19.8 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [6]:
%%timeit -r 10 -n 10
encoder = FastOrdinalEncoder()
encoder.fit(categories=products)
encoded_values = encoder.transform(df[['product']])

385 ms ± 4.6 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [7]:
# Inverse transform

In [8]:
encoder = OrdinalEncoder(
    categories = 'auto',
    dtype      = int
).set_output(transform='pandas')
encoder.fit(df[['product']])
encoded_values = encoder.transform(df[['product']])

In [9]:
%%timeit -r 10 -n 10
encoder.inverse_transform(encoded_values[['product']])

99.3 ms ± 3.59 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [10]:
encoder = FastOrdinalEncoder()
encoder.fit(categories=products)
encoded_values = encoder.transform(df[['product']])

In [11]:
%%timeit -r 10 -n 10
encoder.inverse_transform(encoded_values[['product']])

93.1 ms ± 2.66 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [12]:
encoder = OrdinalEncoder(
    categories = 'auto',
    dtype      = int
).set_output(transform='pandas')
encoder.fit(df[['product']])
encoded_values = encoder.transform(df[['product']])
display(encoded_values)

inverse_values = encoder.inverse_transform(encoded_values[['product']])
display(inverse_values.ravel())

,product
0,0
1,0
2,0
3,0
4,0
...,...
4999995,4447
4999996,4447
4999997,4447
4999998,4447


array(['series_1', 'series_1', 'series_1', ..., 'series_5000',
       'series_5000', 'series_5000'], shape=(5000000,), dtype=object)

In [13]:
encoder = FastOrdinalEncoder()
encoder.fit(categories=products)
encoded_values_fast = encoder.transform(df['product'])
display(encoded_values_fast)
inverse_values_fast = encoder.inverse_transform(encoded_values_fast[['product']])
display(inverse_values_fast.ravel())

,product
0,0
1,0
2,0
3,0
4,0
...,...
4999995,4447
4999996,4447
4999997,4447
4999998,4447


array([np.str_('series_1'), np.str_('series_1'), np.str_('series_1'), ...,
       np.str_('series_5000'), np.str_('series_5000'),
       np.str_('series_5000')], shape=(5000000,), dtype=object)